In [1]:
### Import required libraries
from pylibCZIrw import czi as pyczi
import json
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import os, sys
import time
import glob
from tqdm import tqdm
from tqdm.contrib import itertools as it
from matplotlib.patches import Rectangle
import cv2
from scipy.ndimage import distance_transform_edt
from scipy.stats import mode

from cztile.fixed_total_area_strategy import AlmostEqualBorderFixedTotalAreaStrategy2D
from cztile.tiling_strategy import Rectangle as czrect

# show the used python env
print("Using:", sys.executable)

/mnt/hd1/ani/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using: /mnt/hd1/ani/conda/bin/python


### We want to process all ".czi" files. For each file, we will generate and save 4096x4096 tiles for all "scenes".

In [2]:
# ### Function that takes extracted 4096x4096 region and returns a background matched image
# def bkgrd_match(img, f_rsz=8, sz=25):

#     # Lets reshape the image to (512, 512, 3) to speed-up the algorithm
#     img_f = cv2.resize(img, (int(img.shape[0]/f_rsz), int(img.shape[1]/f_rsz)))
#     # img_f = img

#     ### Trade-off b/w size of neighborhood vs computation time
#     # If we choose a very low neighborhood size, it solves the "tissue" pixel problem but runtime might be too high
                    
#     print(f"Running background matching algorithm ..")

#     t1 = time.time()

#     while np.min(img_f) == 0:

#         # First, lets find all black pixel locations
#         # For every black pixel, consider RxR (3x3) neighborhood. In this region, if highest pixel value > 0, then most probably it is a "true background" pixel.
#         # If we choose R to be too large, there is a chance of picking up "tissue" pixels which we don't want
#         # Lets say, we pick up the right pixels. Then, replace black pixel with mean(background intensity)

#         inds = np.where(img_f[:,:,0] == 0)   # locations of all black pixels

#         for j in range(len(inds[0])):

#             if (inds[0][j] >= sz) & (inds[0][j] < img_f.shape[0]-sz) & (inds[1][j] >= sz) & (inds[1][j] < img_f.shape[1]-sz):
#                 region = img_f[inds[0][j]-sz:inds[0][j]+sz+1, inds[1][j]-sz:inds[1][j]+sz+1, :]

#             if (inds[0][j] < sz) & (inds[1][j] < sz):
#                 region = img_f[:(2*sz)+1, :(2*sz)+1, :]
    
#             if (inds[0][j] < sz) & ((inds[1][j] >= sz) & (inds[1][j] < img_f.shape[1]-sz)):
#                 region = img_f[:(2*sz)+1, inds[1][j]-sz:inds[1][j]+sz+1, :]

#             if (inds[0][j] < sz) & (inds[1][j] >= img_f.shape[1]-sz):
#                 region = img_f[:(2*sz)+1, -((2*sz)+1):, :]

#             if (inds[1][j] < sz) & (inds[0][j] >= sz) & (inds[0][j] < img_f.shape[0]-sz):
#                 region = img_f[inds[0][j]-sz:inds[0][j]+sz+1, :(2*sz)+1, :]

#             if (inds[1][j] < sz) & (inds[0][j] >= img_f.shape[0]-sz):
#                 region = img_f[-((2*sz)+1):, :(2*sz)+1, :]

#             if (inds[0][j] >= img_f.shape[0]-sz) & (inds[1][j] >= sz) & (inds[1][j] < img_f.shape[1]-sz):
#                 region = img_f[-((2*sz)+1):, inds[1][j]-sz:inds[1][j]+sz+1, :]

#             if (inds[0][j] >= img_f.shape[0]-sz) & ((inds[1][j] >= img_f.shape[1]-sz)):
#                 region = img_f[-((2*sz)+1):, -((2*sz)+1):, :]

#             if ((inds[1][j] >= img_f.shape[1]-sz)) & (inds[0][j] >= sz) & (inds[0][j] < img_f.shape[0]-sz):
#                 region = img_f[inds[0][j]-sz:inds[0][j]+sz+1, -((2*sz)+1):, :]

#             # Now, we want to convert the black pixel into the corresponding gray value
#             pix_max = np.max(region)
                            
#             # If all pixels in the "region" are black, we don't want to run anything further
#             if pix_max == 0:
#                 continue

#             region_gray_exists = region[region > 0]   # get all background pixels

#             if len(region_gray_exists) > 0:

#                 # find "true background" pixel locations
#                 inds_gray = np.where(region > 0)   

#                 # mean background pixel value
#                 grays_0, grays_1, grays_2 = np.mean(region[inds_gray[0], inds_gray[1], 0]), np.mean(region[inds_gray[0], inds_gray[1], 1]), np.mean(region[inds_gray[0], inds_gray[1], 2])                
#                 # grays_00, grays_11, grays_22 = np.mean(img[inds_gray[0]*f_rsz, inds_gray[1]*f_rsz, 0]), np.mean(img[inds_gray[0]*f_rsz, inds_gray[1]*f_rsz, 1]), np.mean(img[inds_gray[0]*f_rsz, inds_gray[1]*f_rsz, 2])

#                 # replace black pixels by mean background pixel value
#                 img_f[inds[0][j], inds[1][j], 0], img_f[inds[0][j], inds[1][j], 1], img_f[inds[0][j], inds[1][j], 2] = grays_0, grays_1, grays_2
#                 # img[inds[0][j]*f_rsz, inds[1][j]*f_rsz, 0], img[inds[0][j]*f_rsz, inds[1][j]*f_rsz, 1], img[inds[0][j]*f_rsz, inds[1][j]*f_rsz, 2] = grays_00, grays_11, grays_22


#     # Reshaping back to original size
#     img_f_rshp = cv2.resize(img_f, (img.shape[0], img.shape[1]))
#     # img_f_rshp = img
    
#     t2 = time.time()
#     print(f"Background matching algorithm runtime: {t2-t1} seconds")

#     print("Finished running background matching algorithm!")

#     # # Lets visualize the result
#     # plt.figure(figsize=(12,6))
#     # plt.subplot(121)
#     # plt.imshow(img)
#     # plt.subplot(122)
#     # plt.imshow(img_f_rshp)

#     return img_f_rshp

In [3]:
# Folder where all WSI ".czi" files exist
dir_rawfiles = "/mnt/hd1/ani/Liver_pathology_project/Raw_data/Anh_slides/"

# Folder where we want to save all extracted tiles
dir_imgs = "/mnt/hd1/ani/Liver_pathology_project/Prepared_datasets/Tile_size_4096x4096/Anh_slides/"

# Get list of all patient WSIs
wsi_img_list = (glob.glob(dir_rawfiles + "*.czi"))
# wsi_img_list

# create a "tile" by specifying the desired tile dimension and the
# minimum required overlap between tiles (depends on the processing)
tiler = AlmostEqualBorderFixedTotalAreaStrategy2D(total_tile_width=4096,
                                                  total_tile_height=4096,
                                                  min_border_width=0)

for w in range(0, 1):
    
    # Get WSI filename w/o extension - we will use this to name folders
    filename_noext = os.path.splitext(os.path.basename(wsi_img_list[w]))[0][-2:]
    
    # Create unique folder for patient
    parent_dir = dir_imgs
    directory = filename_noext
    path = os.path.join(parent_dir, directory)        

    if os.path.exists(path) == False:
        
        print(f'Saving WSI: {filename_noext} for quantitative analysis ...')
        os.mkdir(path)
    
        # create CZI instance to read some metadata 
        with pyczi.open_czi(wsi_img_list[w]) as czidoc_r:
    
            # get the bounding boxes for each individual scene: this will be a dictionary with keys = scene indices, values = [x,y,w,h] for that scene
            scenes_bounding_rectangle = czidoc_r.scenes_bounding_rectangle

            for i in range(len(scenes_bounding_rectangle)):

                # Read entire scene as an array
                scene_arr = czidoc_r.read(roi=scenes_bounding_rectangle[i], scene=i, zoom=1.0)
                scene_arr = scene_arr[:, :, ::-1]   # BGR --> RGB

                # get the size of the bounding rectange for the scenes
                tiles = tiler.tile_rectangle(czidoc_r.scenes_bounding_rectangle[i])   # first scene

                for f, tile in enumerate(tiles):

                    # print(f"Started processing scene {i+1}, tile {f+1}")

                    roi_x, roi_y, roi_w, roi_h = tile.roi.x, tile.roi.y, tile.roi.w, tile.roi.h            
                    patch_4k = scene_arr[roi_y-np.abs(scenes_bounding_rectangle[i][1]):roi_y-np.abs(scenes_bounding_rectangle[i][1])+roi_h, roi_x+np.abs(scenes_bounding_rectangle[i][0]):roi_x+np.abs(scenes_bounding_rectangle[i][0])+roi_w, :]
                    
                    # if (i == 0) & (f < 25):

                    # ### Background matching algorithm - iterative correction
                    # patch_4k_f = bkgrd_match(patch_4k, f_rsz=16, sz=25)                       
                        
                    if patch_4k.std() >= 30:

                        plt.imsave(f"/mnt/hd1/ani/Liver_pathology_project/Prepared_datasets/Tile_size_4096x4096/Anh_slides/{directory}/sc_{i}_patch_{f}.png", patch_4k)
                        # plt.imsave(f"/mnt/hd1/ani/Liver_pathology_project/Prepared_datasets/Tile_size_4096x4096/Anh_slides/{directory}/sc_{i}_patch_{f}_post.png", patch_4k_f)
                    
                    # print(f"Finished processing scene {i+1}, tile {f+1}")

        print(f'WSI: {filename_noext} imageset created !')
    else:
        print('Folder already exists !!')

Saving WSI: 26 for quantitative analysis ...


### Testing

In [ ]:
# # Folder where all WSI ".czi" files exist
# dir_rawfiles = "/mnt/hd1/ani/Liver_pathology_project/Raw_data/Anh_slides/"

# # Folder where we want to save all extracted tiles
# dir_imgs = "/mnt/hd1/ani/Liver_pathology_project/Prepared_datasets/Tile_size_4096x4096/Anh_slides/Testing/"

# # Get list of all patient WSIs
# wsi_img_list = (glob.glob(dir_rawfiles + "*.czi"))
# # wsi_img_list

# # create a "tile" by specifying the desired tile dimension and the
# # minimum required overlap between tiles (depends on the processing)
# tiler = AlmostEqualBorderFixedTotalAreaStrategy2D(total_tile_width=4096,
#                                                   total_tile_height=4096,
#                                                   min_border_width=0)

# # Choose "scene" and "tile" to analyze
# sc, ti = 2, 2

# for w in range(5, 6):
    
#     # Get WSI filename w/o extension - we will use this to name folders
#     filename_noext = os.path.splitext(os.path.basename(wsi_img_list[w]))[0][-2:]
    
#     # Create unique folder for patient
#     parent_dir = dir_imgs
#     directory = filename_noext
#     path = os.path.join(parent_dir, directory)        
            
#     # create CZI instance to read some metadata 
#     with pyczi.open_czi(wsi_img_list[w]) as czidoc_r:
    
#         # get the bounding boxes for each individual scene: this will be a dictionary with keys = scene indices, values = [x,y,w,h] for that scene
#         scenes_bounding_rectangle = czidoc_r.scenes_bounding_rectangle

#         for i in range(len(scenes_bounding_rectangle)):
            
#             if i == sc+1:
#                 break

#             # Read entire scene as an array
#             scene_arr = czidoc_r.read(roi=scenes_bounding_rectangle[i], scene=i, zoom=1.0)
#             scene_arr = scene_arr[:, :, ::-1]   # BGR --> RGB

#             # get the size of the bounding rectange for the scenes
#             tiles = tiler.tile_rectangle(czidoc_r.scenes_bounding_rectangle[i])   # first scene

#             for f, tile in enumerate(tiles):

#                 if f == ti+1:
#                     break

#                 # print(f"Started processing scene {i+1}, tile {f+1}")

#                 roi_x, roi_y, roi_w, roi_h = tile.roi.x, tile.roi.y, tile.roi.w, tile.roi.h            
#                 patch_4k = scene_arr[roi_y-np.abs(scenes_bounding_rectangle[i][1]):roi_y-np.abs(scenes_bounding_rectangle[i][1])+roi_h, roi_x+np.abs(scenes_bounding_rectangle[i][0]):roi_x+np.abs(scenes_bounding_rectangle[i][0])+roi_w, :]
                    
#                 if patch_4k.std() >= 30:

#                     if (i == sc) & (f == ti):

#                         ### Background matching algorithm - iterative correction
#                         patch_4k_f = bkgrd_match(patch_4k, f_rsz=1, sz=25)    

#                         # plt.imsave(f"/mnt/hd1/ani/Liver_pathology_project/Prepared_datasets/Tile_size_4096x4096/Anh_slides/Testing/sc_{i}_patch_{f}_pre.png", patch_4k)
#                         plt.imsave(f"/mnt/hd1/ani/Liver_pathology_project/Prepared_datasets/Tile_size_4096x4096/Anh_slides/Testing/sc_{i}_patch_{f}_ds_fact_1_sz_25_post.png", patch_4k_f)
                                       
#                 # print(f"Finished processing scene {i+1}, tile {f+1}")